# Download Video Info for The Needle Drop

https://www.youtube.com/c/theneedledrop

In [2]:
%run bootstrap.ipynb

In [3]:
import json
import itertools

import sqlite3

import pandas as pd

from IPython.display import display, JSON, HTML

from castle.music_meta.youtube import YouTubeClient

## Config

In [4]:
username = "theneedledrop"

## Setup DB Connection

In [5]:
db = sqlite3.Connection("var/data/music.db")

## Setup YouTube Client

In [6]:
yt_client = YouTubeClient(
    CONFIG["youtube"]["base-url"],
    CONFIG["youtube"]["api-key"]
)

yt_client

## Download Video Info

In [7]:
# Get the upload playlist for needle drop
playlist_id = yt_client.get_upload_playlist_id(username)
playlist_id

'UUt7fwAhXDy3oNFTAzF2o8Pw'

In [8]:
# TODO: Enhance the API to pass in last known upload date for a video to get only new ones.

videos_iter = yt_client.enumerate_playlist_items(playlist_id)
videos = list(videos_iter)

In [15]:
# Extract Fields And Put Into DataFrame

def extract_video_fields(video):
    return {
        'channelTitle': video['snippet']['channelTitle'],
        'videoId': video['contentDetails']['videoId'],
        'videoPublishedAt': video['contentDetails']['videoPublishedAt'],
        'title': video['snippet']['title'],
        'description': video['snippet']['description']
    }
    
video_info = pd.DataFrame([extract_video_fields(v) for v in videos])

## Save Video Info to Database

In [24]:
video_info.to_sql('yt_video_info', db, if_exists='append', index=False)

In [32]:
# Create a couple of indices

cur = db.cursor()
cur.execute(f"""
  create unique index if not exists i_yt_video_info_pk on yt_video_info (videoId)
""")
cur.execute(f"""
  create index if not exists i_yt_video_info_channelTitle on yt_video_info (channelTitle)
""")
cur.close()

## Test Reading Back From DB

In [31]:
pd.read_sql("""
  select *
  from yt_video_info
  where channelTitle = 'theneedledrop' and title like '%ALBUM%'
""", db)

,channelTitle,videoId,videoPublishedAt,title,description
0,theneedledrop,3VOAXpSR9V0,2021-11-06T05:13:32Z,Marissa Nadler - The Path of the Clouds ALBUM ...,Listen: https://marissanadler.bandcamp.com/alb...
1,theneedledrop,EZc4QCJmKdQ,2021-11-05T02:21:08Z,The War on Drugs - I Don't Live Here Anymore A...,Listen: https://www.youtube.com/watch?v=B3OEof...
2,theneedledrop,eBgdS1yHwRk,2021-11-04T03:24:04Z,Mastodon - Hushed and Grim ALBUM REVIEW,Listen: https://www.youtube.com/watch?v=haT5ar...
3,theneedledrop,aae1g4R9KJA,2021-11-02T02:11:18Z,Ed Sheeran - = ALBUM REVIEW,Listen: https://www.youtube.com/watch?v=orJSJG...
4,theneedledrop,bbsD3mRmQn4,2021-10-29T04:05:54Z,JPEGMAFIA - LP! ALBUM REVIEW,Listen: https://jpegmafia.bandcamp.com/album/l...
...,...,...,...,...,...
2461,theneedledrop,DuMUDldrG3g,2010-05-12T05:43:37Z,The National- High Violet ALBUM REVIEW,Review: http://bit.ly/95B1DQ / http://bit.ly/c...
2462,theneedledrop,ySXryTlo9Ac,2010-05-10T19:12:42Z,Holy Fuck- Latin ALBUM REVIEW,Listen: http://bit.ly/cE69Wl / http://bit.ly/c...
2463,theneedledrop,cndwH6byJnk,2010-05-09T18:18:46Z,Little Women- Throat ALBUM REVIEW,Listen: http://www.myspace.com/littlewomensoun...
2464,theneedledrop,KCuamde9Atc,2010-05-05T18:47:37Z,Flying Lotus- Cosmogramma ALBUM REVIEW,Listen: http://bit.ly/cY4EPL / http://bit.ly/a...
